<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Regression Models with Keras</font></h1>

## Introduction

As we discussed in the videos, despite the popularity of more powerful libraries such as PyTorch and TensorFlow, they are not easy to use and have a steep learning curve. So, for people who are just starting to learn deep learning, there is no better library to use other than the Keras library. 

Keras is a high-level API for building deep learning models. It has gained favor for its ease of use and syntactic simplicity facilitating fast development. As you will see in this lab and the other labs in this course, building a very complex deep learning network can be achieved with Keras with only few lines of code. You will appreciate Keras even more, once you learn how to build deep models using PyTorch and TensorFlow in the other courses.

So, in this lab, you will learn how to use the Keras library to build a regression model.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Download and Clean Dataset</a>  
2. <a href="#item32">Import Keras</a>  
3. <a href="#item33">Build a Neural Network</a>  
4. <a href="#item34">Train and Test the Network</a>  

</font>
</div>

<a id="item31"></a>

## Download and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's download the data and read it into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [10]:
n_cols = predictors_norm.shape[1] # number of predictors

In [16]:
predictors_norm.shape

(1030, 8)

<a id="item1"></a>

<a id='item32'></a>

## Import Keras

Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.

#### Let's go ahead and import the Keras library

In [18]:
import keras

Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py

As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [20]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [21]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.

<a id="item4"></a>

<a id='item34'></a>

## Train and Test the Network

Let's call the function now to create our model.

In [22]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.

In [24]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=1000, verbose=2)

Train on 721 samples, validate on 309 samples
Epoch 1/1000
 - 0s - loss: 34.9719 - val_loss: 174.0151
Epoch 2/1000
 - 0s - loss: 34.6509 - val_loss: 173.1583
Epoch 3/1000
 - 0s - loss: 34.4140 - val_loss: 183.5618
Epoch 4/1000
 - 0s - loss: 34.1572 - val_loss: 181.2224
Epoch 5/1000
 - 0s - loss: 34.0216 - val_loss: 175.8781
Epoch 6/1000
 - 0s - loss: 33.6800 - val_loss: 167.0764
Epoch 7/1000
 - 0s - loss: 32.6363 - val_loss: 172.8438
Epoch 8/1000
 - 0s - loss: 32.4172 - val_loss: 179.1958
Epoch 9/1000
 - 0s - loss: 32.1895 - val_loss: 164.1291
Epoch 10/1000
 - 0s - loss: 31.5240 - val_loss: 173.7228
Epoch 11/1000
 - 0s - loss: 31.4052 - val_loss: 175.6320
Epoch 12/1000
 - 0s - loss: 31.0202 - val_loss: 157.7795
Epoch 13/1000
 - 0s - loss: 31.1234 - val_loss: 166.7478
Epoch 14/1000
 - 0s - loss: 30.5934 - val_loss: 173.5571
Epoch 15/1000
 - 0s - loss: 29.9857 - val_loss: 159.6528
Epoch 16/1000
 - 0s - loss: 32.0811 - val_loss: 153.9462
Epoch 17/1000
 - 0s - loss: 30.3668 - val_loss: 178

Epoch 144/1000
 - 0s - loss: 14.8204 - val_loss: 145.4845
Epoch 145/1000
 - 0s - loss: 15.2953 - val_loss: 150.2830
Epoch 146/1000
 - 0s - loss: 14.8143 - val_loss: 150.4758
Epoch 147/1000
 - 0s - loss: 14.4426 - val_loss: 143.6624
Epoch 148/1000
 - 0s - loss: 14.5373 - val_loss: 133.7220
Epoch 149/1000
 - 0s - loss: 15.1055 - val_loss: 139.7103
Epoch 150/1000
 - 0s - loss: 14.8145 - val_loss: 138.4347
Epoch 151/1000
 - 0s - loss: 14.3117 - val_loss: 148.8110
Epoch 152/1000
 - 0s - loss: 14.5406 - val_loss: 151.5883
Epoch 153/1000
 - 0s - loss: 14.3699 - val_loss: 138.1402
Epoch 154/1000
 - 0s - loss: 14.2632 - val_loss: 141.6500
Epoch 155/1000
 - 0s - loss: 14.4037 - val_loss: 140.8125
Epoch 156/1000
 - 0s - loss: 15.2093 - val_loss: 156.2251
Epoch 157/1000
 - 0s - loss: 14.3879 - val_loss: 149.1002
Epoch 158/1000
 - 0s - loss: 14.2765 - val_loss: 135.5601
Epoch 159/1000
 - 0s - loss: 14.2829 - val_loss: 148.8461
Epoch 160/1000
 - 0s - loss: 14.2654 - val_loss: 143.8028
Epoch 161/1000

Epoch 286/1000
 - 0s - loss: 10.6588 - val_loss: 155.1933
Epoch 287/1000
 - 0s - loss: 10.8190 - val_loss: 141.8201
Epoch 288/1000
 - 0s - loss: 10.9299 - val_loss: 157.3097
Epoch 289/1000
 - 0s - loss: 11.1421 - val_loss: 151.0949
Epoch 290/1000
 - 0s - loss: 10.9065 - val_loss: 158.1975
Epoch 291/1000
 - 0s - loss: 10.3174 - val_loss: 147.8049
Epoch 292/1000
 - 0s - loss: 10.6670 - val_loss: 161.8678
Epoch 293/1000
 - 0s - loss: 11.3724 - val_loss: 146.5358
Epoch 294/1000
 - 0s - loss: 11.0343 - val_loss: 147.8963
Epoch 295/1000
 - 0s - loss: 10.8502 - val_loss: 144.0228
Epoch 296/1000
 - 0s - loss: 10.7009 - val_loss: 144.4775
Epoch 297/1000
 - 0s - loss: 10.3900 - val_loss: 159.1030
Epoch 298/1000
 - 0s - loss: 10.4880 - val_loss: 149.4383
Epoch 299/1000
 - 0s - loss: 10.1835 - val_loss: 149.2071
Epoch 300/1000
 - 0s - loss: 10.2803 - val_loss: 152.4781
Epoch 301/1000
 - 0s - loss: 10.2283 - val_loss: 147.5182
Epoch 302/1000
 - 0s - loss: 10.3597 - val_loss: 156.3034
Epoch 303/1000

Epoch 429/1000
 - 0s - loss: 8.9788 - val_loss: 164.5368
Epoch 430/1000
 - 0s - loss: 8.7291 - val_loss: 165.4942
Epoch 431/1000
 - 0s - loss: 9.0011 - val_loss: 159.3871
Epoch 432/1000
 - 0s - loss: 8.5207 - val_loss: 157.7935
Epoch 433/1000
 - 0s - loss: 8.7321 - val_loss: 158.3256
Epoch 434/1000
 - 0s - loss: 8.6719 - val_loss: 159.2968
Epoch 435/1000
 - 0s - loss: 8.7636 - val_loss: 171.0721
Epoch 436/1000
 - 0s - loss: 8.9911 - val_loss: 156.3133
Epoch 437/1000
 - 0s - loss: 8.9349 - val_loss: 160.3358
Epoch 438/1000
 - 0s - loss: 8.7242 - val_loss: 156.9728
Epoch 439/1000
 - 0s - loss: 8.6727 - val_loss: 158.8602
Epoch 440/1000
 - 0s - loss: 8.3458 - val_loss: 170.7416
Epoch 441/1000
 - 0s - loss: 8.6217 - val_loss: 170.7776
Epoch 442/1000
 - 0s - loss: 8.4297 - val_loss: 165.0177
Epoch 443/1000
 - 0s - loss: 8.5902 - val_loss: 170.8789
Epoch 444/1000
 - 0s - loss: 8.5213 - val_loss: 159.5942
Epoch 445/1000
 - 0s - loss: 9.0985 - val_loss: 158.5877
Epoch 446/1000
 - 0s - loss: 8.

Epoch 573/1000
 - 0s - loss: 7.7880 - val_loss: 180.4408
Epoch 574/1000
 - 0s - loss: 7.6232 - val_loss: 170.8171
Epoch 575/1000
 - 0s - loss: 7.2447 - val_loss: 167.5639
Epoch 576/1000
 - 0s - loss: 7.3558 - val_loss: 169.3974
Epoch 577/1000
 - 0s - loss: 7.3032 - val_loss: 170.7235
Epoch 578/1000
 - 0s - loss: 7.2738 - val_loss: 170.1004
Epoch 579/1000
 - 0s - loss: 7.4742 - val_loss: 160.8436
Epoch 580/1000
 - 0s - loss: 7.3874 - val_loss: 159.2607
Epoch 581/1000
 - 0s - loss: 7.0535 - val_loss: 172.2399
Epoch 582/1000
 - 0s - loss: 7.1792 - val_loss: 166.1423
Epoch 583/1000
 - 0s - loss: 7.3688 - val_loss: 178.3096
Epoch 584/1000
 - 0s - loss: 7.5501 - val_loss: 173.3626
Epoch 585/1000
 - 0s - loss: 7.1241 - val_loss: 171.9074
Epoch 586/1000
 - 0s - loss: 7.1304 - val_loss: 166.6856
Epoch 587/1000
 - 0s - loss: 7.1750 - val_loss: 161.3198
Epoch 588/1000
 - 0s - loss: 7.1082 - val_loss: 164.8673
Epoch 589/1000
 - 0s - loss: 7.0232 - val_loss: 164.8662
Epoch 590/1000
 - 0s - loss: 7.

Epoch 717/1000
 - 0s - loss: 6.5911 - val_loss: 181.0606
Epoch 718/1000
 - 0s - loss: 6.6915 - val_loss: 180.6221
Epoch 719/1000
 - 0s - loss: 6.7293 - val_loss: 170.2459
Epoch 720/1000
 - 0s - loss: 6.7347 - val_loss: 176.1011
Epoch 721/1000
 - 0s - loss: 6.9864 - val_loss: 171.9366
Epoch 722/1000
 - 0s - loss: 6.4585 - val_loss: 188.6891
Epoch 723/1000
 - 0s - loss: 6.6902 - val_loss: 180.0141
Epoch 724/1000
 - 0s - loss: 6.6415 - val_loss: 167.0834
Epoch 725/1000
 - 0s - loss: 6.5504 - val_loss: 186.3907
Epoch 726/1000
 - 0s - loss: 6.3198 - val_loss: 177.0110
Epoch 727/1000
 - 0s - loss: 6.3510 - val_loss: 175.6110
Epoch 728/1000
 - 0s - loss: 6.9518 - val_loss: 176.1969
Epoch 729/1000
 - 0s - loss: 6.6759 - val_loss: 177.7651
Epoch 730/1000
 - 0s - loss: 6.6231 - val_loss: 173.4332
Epoch 731/1000
 - 0s - loss: 6.4292 - val_loss: 182.8087
Epoch 732/1000
 - 0s - loss: 6.5887 - val_loss: 181.6929
Epoch 733/1000
 - 0s - loss: 6.8067 - val_loss: 179.7989
Epoch 734/1000
 - 0s - loss: 6.

Epoch 861/1000
 - 0s - loss: 6.5006 - val_loss: 175.9110
Epoch 862/1000
 - 0s - loss: 5.8298 - val_loss: 177.3379
Epoch 863/1000
 - 0s - loss: 5.6549 - val_loss: 178.6045
Epoch 864/1000
 - 0s - loss: 6.0060 - val_loss: 180.5573
Epoch 865/1000
 - 0s - loss: 5.8743 - val_loss: 174.1246
Epoch 866/1000
 - 0s - loss: 6.0243 - val_loss: 180.2216
Epoch 867/1000
 - 0s - loss: 6.5519 - val_loss: 171.4732
Epoch 868/1000
 - 0s - loss: 5.9710 - val_loss: 183.8663
Epoch 869/1000
 - 0s - loss: 5.8972 - val_loss: 174.0950
Epoch 870/1000
 - 0s - loss: 6.0854 - val_loss: 185.5655
Epoch 871/1000
 - 0s - loss: 6.4327 - val_loss: 166.2328
Epoch 872/1000
 - 0s - loss: 6.5268 - val_loss: 185.9434
Epoch 873/1000
 - 0s - loss: 5.9922 - val_loss: 181.6919
Epoch 874/1000
 - 0s - loss: 5.8311 - val_loss: 181.8089
Epoch 875/1000
 - 0s - loss: 5.5911 - val_loss: 174.7338
Epoch 876/1000
 - 0s - loss: 6.0881 - val_loss: 180.8567
Epoch 877/1000
 - 0s - loss: 5.9480 - val_loss: 175.9242
Epoch 878/1000
 - 0s - loss: 5.

<strong>You can refer to this [link](https://keras.io/models/sequential/) to learn about other functions that you can use for prediction or evaluation.</strong>

Feel free to vary the following and note what impact each change has on the model's performance:

1. Increase or decreate number of neurons in hidden layers
2. Add more hidden layers
3. Increase number of epochs

### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!

This notebook is part of a course on **Coursera** called *Introduction to Deep Learning & Neural Networks with Keras*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0101EN_Coursera_Week3_LAB1).

<hr>

Copyright &copy; 2019 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).